# Debug centering issue

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
from pastis.e2e_simulators.indexed_segmented_mirror import SegmentedMirror

import pastis.util as util
from pastis.e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
# Instantiate LUVOIR
apodizer_design = 'small'
sampling = 4
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image
luvoir.flatten()
psf_unaber, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield')

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

In [ ]:
inter.keys()

In [ ]:
to_plot = inter['before_lyot']

In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(2, 2, 1)
plt.title("Intensity")
hc.imshow_field(np.log10(to_plot.intensity))

plt.subplot(2, 2, 2)
plt.title("Phase")
hc.imshow_field(to_plot.phase)

plt.subplot(2, 2, 3)
plt.title("Real")
hc.imshow_field(to_plot.real)

plt.subplot(2, 2, 4)
plt.title("Imaginary")
hc.imshow_field(to_plot.imag)

In [ ]:
(np.arange(10) + 0.5 - 10/2)

In [ ]:
plt.figure(figsize=(10,10))
hc.imshow_field(luvoir.fpm)
#plt.grid(color='w', linestyle='-', linewidth=2)

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(luvoir.fpm.shaped)

In [ ]:
print(luvoir.fpm.shaped)
util.write_fits(luvoir.fpm.shaped, '/Users/ilaginja/Documents/fpm.fits')

In [ ]:
res = util.FFT(luvoir.fpm.shaped)

In [ ]:
new_plot = res

im = util.zoom_point(new_plot, new_plot.shape[0]/2, new_plot.shape[0]/2, 200)

In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(2, 2, 1)
plt.title("Intensity")
plt.imshow(np.log10(np.abs(new_plot)**2))

plt.subplot(2, 2, 2)
plt.title("Phase")
plt.imshow(np.angle(new_plot))
plt.colorbar()

plt.subplot(2, 2, 3)
plt.title("Real")
#hc.imshow_field(to_plot.real)

plt.subplot(2, 2, 4)
plt.title("Imaginary")
#hc.imshow_field(to_plot.imag)

In [ ]:
np.angle(new_plot)

In [ ]:
print(np.min(np.angle(new_plot)))

In [ ]:
# Plot
plt.figure(figsize=(18, 6))
plt.subplot(131)
hc.imshow_field(psf_unaber.intensity/ref.intensity.max(), norm=LogNorm())
plt.subplot(132)
hc.imshow_field(dh_mask)
plt.subplot(133)
hc.imshow_field(psf_unaber.intensity/ref.intensity.max(), norm=LogNorm(), mask=dh_mask)

In [ ]:
dh_intensity = psf_unaber.intensity/ref.intensity.max() * dh_mask
baseline_contrast = util.dh_mean(dh_intensity, dh_mask)
#np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('Baseline contrast:', baseline_contrast)

In [ ]:
imsize = 10
im = np.zeros((imsize, imsize))
focal_plane_mask = util.circle_mask(im, imsize/2, imsize/2, imsize/2)

In [ ]:
plt.imshow(focal_plane_mask)

In [ ]:
out = util.FFT(focal_plane_mask)

In [ ]:
plt.imshow(np.abs(out))

In [ ]:
plt.imshow(np.angle(out))

In [ ]:
np.angle(out)